In [ ]:
# Install essentials
!pip install -U uv
!git clone https://github.com/Zyphra/Zonos.git
%cd Zonos
!uv sync --extra compile  # For hybrid model compilation
!apt install -y espeak-ng  # For phonemization
!pip install pdfplumber pydub deepgram-sdk torchaudio torch
print("Dependencies installed.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 101.6 MB/s eta 0:00:00
Cloning into 'Zonos'...
remote: Enumerating objects: 340, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 340 (delta 159), reused 116 (delta 116), pack-reused 144 (from 1)
Receiving objects: 100% (340/340), 3.03 MiB | 6.86 MiB/s, done.
Resolving deltas: 100% (211/211), done.
/content/Zonos/Zonos
Using CPython 3.12.11 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
Resolved 116 packages in 1ms
Prepared 110 packages in 1m 32s
Installed 110 packages in 712ms
Prepared 3 packages without build isolation in 33.73s
Installed 3 packages in 4ms
 + aiofiles==23.2.1
 + annotated-types==0.7.0
 + anyio==4.8.0
 + attrs==25.1.0
 + babel==2.17.0
 + causal-conv1d==1.5.0.post8
 + certifi==2025.1.31
 + cffi==1.17.1
 + charset-normalizer==3.4.1
 + click==8.1.8
 + clldutils==3.21.0
 + colorama==0.4.6
 + colorlog==6.9.0
 + csvw==3.5.1
 +

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Upload PDF and voice sample
print("Upload your 'Life 3.0' PDF and your voice sample (.wav recommended):")
uploaded = files.upload()
pdf_path = next((k for k in uploaded if '.pdf' in k), 'life_3_0.pdf')
voice_path = next((k for k in uploaded if k.endswith(('.wav', '.mp3'))), 'my_voice.wav')
print(f"PDF: {pdf_path}, Voice: {voice_path}")





# Check if voice_path is defined and exists as a file
print(f"PDF: {pdf_path}, Voice: {voice_path}")
if voice_path and os.path.exists(voice_path):
    print(f"'{voice_path}' exists and is ready for use.")
else:
    print(f"'{voice_path}' does not exist or is not defined. Please re-upload a .wav or .mp3 file.")

Upload your 'Life 3.0' PDF and your voice sample (.wav recommended):


Saving myvoice.mp3 to myvoice (1).mp3
Saving max-tegmark-life-30-being-human-in-the-age-of-artificial-intelligence-alfred-a-knopf-2017-aTvn.pdf to max-tegmark-life-30-being-human-in-the-age-of-artificial-intelligence-alfred-a-knopf-2017-aTvn (1).pdf
PDF: max-tegmark-life-30-being-human-in-the-age-of-artificial-intelligence-alfred-a-knopf-2017-aTvn (1).pdf, Voice: myvoice (1).mp3
PDF: max-tegmark-life-30-being-human-in-the-age-of-artificial-intelligence-alfred-a-knopf-2017-aTvn (1).pdf, Voice: myvoice (1).mp3
'myvoice (1).mp3' exists and is ready for use.


In [ ]:
# Enter Deepgram API Key
DEEPGRAM_API_KEY = input("Enter your Deepgram API Key: ")
print("API key entered.")


Enter your Deepgram API Key: 6b287043571893a4c9c0cacd166076e47d319e89
API key entered.


In [ ]:
import pdfplumber

page_range = None  # Example: '1-10', or None for full book

def extract_text(pdf_path, page_range=None):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        start, end = 0, len(pdf.pages)
        if page_range:
            s, e = page_range.split('-')
            start, end = int(s) - 1, int(e)
        for page in pdf.pages[start:end]:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

book_text = extract_text(pdf_path, page_range)
print(f"Extracted {len(book_text)} characters from book.")


Extracted 735382 characters from book.


In [ ]:
from deepgram import DeepgramClient, SpeakOptions
from pydub import AudioSegment
import os

deepgram = DeepgramClient(DEEPGRAM_API_KEY)

# Split into chunks for Deepgram
chunks = [book_text[i:i+2000] for i in range(0, len(book_text), 2000)]
deepgram_chunks = []

for idx, chunk in enumerate(chunks):
    print(f"Processing chunk {idx+1}/{len(chunks)}...")

    options = SpeakOptions(model="aura-asteria-en", encoding="linear16", container="wav")
    response = deepgram.speak.v("1").stream_memory({"text": chunk}, options)

    temp_path = f"deepgram_chunk_{idx}.wav"
    with open(temp_path, "wb") as f:
        f.write(response.stream.read())   # save temp wav

    deepgram_chunks.append(temp_path)

# Merge into one wav
combined_deepgram = AudioSegment.empty()
for path in deepgram_chunks:
    combined_deepgram += AudioSegment.from_wav(path)
    os.remove(path)  # cleanup temp files

deepgram_wav = "/content/drive/My Drive/life_30_deepgram_tts.wav"
combined_deepgram.export(deepgram_wav, format="wav")
print(f"✅ Deepgram TTS saved to: {deepgram_wav}")


Processing chunk 1/368...


/tmp/ipython-input-53451939.py:15: UnsupportedWarning: v is unsupported as of 4.0.0. deepgram.speak.v1 is deprecated. Use deepgram.speak.rest or deepgram.speak.websocket instead.
  response = deepgram.speak.v("1").stream_memory({"text": chunk}, options)


Processing chunk 2/368...
Processing chunk 3/368...
Processing chunk 4/368...
Processing chunk 5/368...
Processing chunk 6/368...
Processing chunk 7/368...
Processing chunk 8/368...
Processing chunk 9/368...
Processing chunk 10/368...
Processing chunk 11/368...
Processing chunk 12/368...
Processing chunk 13/368...
Processing chunk 14/368...
Processing chunk 15/368...
Processing chunk 16/368...
Processing chunk 17/368...
Processing chunk 18/368...
Processing chunk 19/368...
Processing chunk 20/368...
Processing chunk 21/368...
Processing chunk 22/368...
Processing chunk 23/368...
Processing chunk 24/368...
Processing chunk 25/368...
Processing chunk 26/368...
Processing chunk 27/368...
Processing chunk 28/368...
Processing chunk 29/368...
Processing chunk 30/368...
Processing chunk 31/368...
Processing chunk 32/368...
Processing chunk 33/368...
Processing chunk 34/368...
Processing chunk 35/368...
Processing chunk 36/368...
Processing chunk 37/368...
Processing chunk 38/368...
Processin